# 16 有监督学习 —— 基于朴素贝叶斯方法的文本分类

<img src="images\贝叶斯.jfif" width="300" alt="" hspace="4">

- 托马斯·贝叶斯（Thomas Bayes，1702-1761），英国神学家、数学家、数理统计学家和哲学家
    - 生前并无论文著作发表，死后 (1763) 由朋友在笔记遗物中发现贝叶斯定理，并发表
    - 今天，人们认为他是
        - 概率论理论创始人之一
        - 贝叶斯统计的创立者
        - “归纳地”运用数学概率
        - <font color="red">实现逆概率计算</font>
        - “从特殊推论一般、从样本推论全体”的第一人


- 朴素贝叶斯
    - 基于贝叶斯定理的一类简单概率分类器


- 广泛应用于
    - 文本分类
    - 垃圾邮件过滤
    - 自然语言处理 (Natural Language Processing, NLP)
    - 等

## 16.1 基本概率公式、贝叶斯定理

### 16.1.1 基本概率公式

#### (1) 条件概率公式

$$
P(A|B)=\dfrac{P(A\bigcap B)}{P(B)}=\dfrac{P(AB)}{P(B)}
$$

#### (2) 乘法公式

$$
P(AB) = {P(A)}P(B|A) = {P(B)}P(A|B)
$$

#### (3) 全概率公式

$$
P(A) = \sum_i{P(AB_i)} = \sum_i{P(B_i)P(A|B_i)}
$$

### 16.1.2 贝叶斯定理


- 意义
    - 实现逆概率的计算

#### (1) 基本形式

- 关于随机事件 $A$ 和 $B$ 的条件概率的一则定理

$$
P(A|B)=\dfrac{P(A)\times P(B|A)}{P(B)}
$$


- 其中
    - $P(A|B)$ —— $B$ 发生后，$A$ 发生的<font color="red">条件概率</font>，也称之为<font color="red"> $A$ 的后验概率</font>(先有 $B$ 的概率取值)
    - $P(B|A)$ —— $A$ 发生后，$B$ 发生的<font color="red">条件概率</font>，也称之为<font color="red"> $B$ 的后验概率</font>(先有 $A$ 的概率取值)
    - $P(A)$ —— <font color="red">$A$ 的先验概率</font>
    - $P(B)$ —— <font color="red">$B$ 的先验概率</font>
    - $\dfrac{P(B|A)}{P(B)}$ —— <font color="red">标准相似度</font>，调整因子，用于保证预测概率更接近真实


- 另一表述

$$\text{后验概率} = \text{标准相似度} \times \text{先验概率}$$

#### (2) 其它形式

- 设 $A_i$ 是部分集合，有

$$
P(A_i|B)=\dfrac{P(A_i)\times P(B|A_i)}{\sum\limits_{j}{P(B|A_j)P(A_j)}}
$$

#### (3) 先验概率与后验概率的区分 —— 贝叶斯定义的价值


- 在贝叶斯定理出现前，人们
    - 知道 $P(A)$ 和 $P(B)$ 可能不同
    - 但不能区分 $P(B|A)$ 和 $P(A|B)$
    - 甚至认为二者一样

#### (4) 示例 —— 假阳性问题

- 某种疾病的得病概率为 $P(A)=0.001$
- 采用一种检验手段
    - 患者确实得病时，检测阳性的概率为 $P(B|A)=0.99$
    - 患者未得病时，检测阳性的概率为 $P(B|A^c)=0.05$
- 现检测结果为阳性 ($B$)
- 问患者得病的概率 $P(A|B)$ 为多少，不得病的概率 $P(A^c|B)$ 又为多少？

- 答案

$$P(A|B)=\dfrac{P(B|A)\times P(A)}{P(B|A)\times P(A)+P(B|A^c)\times P(A^c)}=\dfrac{0.99\times 0.001}{0.99\times 0.001 + 0.05\times 0.999}=0.019$$

$$P(A^c|B)=\dfrac{P(B|A^c)\times P(A^c)}{P(B|A)\times P(A)+P(B|A^c)\times P(A^c)}=\dfrac{0.05\times 0.999}{0.99\times 0.001 + 0.05\times 0.999}=0.981$$

- $P(B|A)$ 和 $P(A|B)$ 相差很大
- $P(B|A^c)$ 和 $P(A^c|B)$ 也相差很大

## 16.2 朴素贝叶斯 (Naive Bayes) 原理

### 16.2.1 引言


- 简介
    - 利用概率统计知识进行分类的算法
    - 可与决策树、神经网络算法等相媲美


- 假设
    - 某属性值对给定类的影响独立于其它属性值
    - 或称 <font color="red">属性相互独立</font>

### 16.2.2 简介 —— 朴素贝叶斯的概率模型与分类算法


- 数据
    - 单个样本记录 $S^{(j)} = (f^{(j)}_1, f^{(j)}_2, \cdots, f^{(j)}_n)$，其中每个 $f^{(j)}_i\,\,(i=1,2,\cdots, n)$ 为一个特征属性


- 目标
    - 目标分类集合 $\{C_1, C_2, \cdots, C_m\}$，每个样本 $S^{(j)}$ 的目标值 $c^{(j)}$ 都属于其中的某个分类 


- 数据集 —— 全部数据
$$
\begin{array}{cccc|c}
 &  & X & & y \\
\hline
f^{(1)}_1,& f^{(1)}_2,& \cdots,& f^{(1)}_n & c^{(1)}\\
f^{(2)}_1,& f^{(2)}_2,& \cdots,& f^{(2)}_n & c^{(2)}\\
\vdots & \vdots & \ddots & \vdots & \vdots \\
f^{(N)}_1,& f^{(N)}_2,& \cdots,& f^{(N)}_n & c^{(N)}
\end{array}
$$

- 问题
    - 给定：训练样本集合 $\{S^{(1)}, S^{(2)}, \cdots, S^{(N)}\}$、对应的目标集合 $y$
    - 训练：计算出所有的 $P(f_i|C_k), (i=1,2,\cdots, n;\,k=1,2,\cdots, m)$
    - 预测：$S$ 属于何分类？
    
    
- 解法
    - 寻找 $k$，使
$$P(C_k|S) = \max\{P(C_1|S), P(C_2|S), \cdots, P(C_n|S)\}\\
$$

    - 则样本 $S$ 属于分类 $C_k$ （取概率最大的那一个）


- 进一步思考
    - 上述不仅给出了分类，还给出了离散概率分布性质 $$P(C_k|S), k=1,2,\cdots,n$$

### 16.2.3 朴素贝叶斯算法的优缺点


- 优点
    - 方法简单
    - 分类准确率高
    - 速度快


- 缺点
    - 如果属性相互独立的假定失真，分类计算的准确率将下降

## 16.3 国际标准数据集 —— 20-新闻组

### 16.3.1 20-新闻组数据集


- 来历
    - 1995 年，Ken Lang 发布在第十二届国际机器学习会议论文集上，相关论文 《Newsweeder: Learning to filter netnews》


- 数据集组成
    - 20 个主题
    - 近 20000 个新闻组


- 用途
    - 文本分类
    - 文本聚类
    - 等

### 16.3.2 数据源


- 数据源
    - [http://qwone.com/~jason/20Newsgroups/](http://qwone.com/~jason/20Newsgroups/)
    - [http://www.cs.cmu.edu/afs/cs/project/theo-11/www/naive-bayes.html](http://www.cs.cmu.edu/afs/cs/project/theo-11/www/naive-bayes.html)
    - [http://www.cs.cmu.edu/~TextLearning/](http://www.cs.cmu.edu/~TextLearning/)
    - [http://kdd.ics.uci.edu/databases/20newsgroups/20newsgroups.html](http://kdd.ics.uci.edu/databases/20newsgroups/20newsgroups.html)


- 三个网络版本的数据压缩文件
    - [原始数据集 19997](http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz)
    - [按日期排序并去重复 18846](http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz) —— 推荐
    - [去重复 18828](http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz)




### 16.3.3 数据组织 —— 20 个主题


- comp 系列 5 个
    - comp.graphics
    - comp.os.ms-windows.misc
    - comp.sys.ibm.pc.hardware
    - comp.sys.mac.hardware
    - comp.windows.x



- rec 系列 4 个
    - rec.autos (赛车运动)
    - rec.motorcycles (摩托车运动)
    - rec.sport.baseball (棒球运动)
    - rec.sport.hockey (曲棍球运动)


- sci 系列 4 个
    - sci.crypt
    - sci.electronics
    - sci.med
    - sci.space


- misc 系列 1 个
    - misc.forsale (销售)



- talk 系列 3 个
    - talk.politics.misc
    - talk.politics.guns
    - talk.politics.mideast



- 其它 3 个 
    - talk.religion.misc (宗教)
    - alt.atheism (无神论)
    - soc.religion.christian (基督)

## 16.4 数据准备

### 16.4.1 前言


- 朴素贝叶斯方法最成功的应用之一
    - 自然语言处理 `NLP`


- 本节实例 —— 新闻组信息识别
    - 数据集：已分好类的文本集
    - 任务：训练机器来预测新实例

### 16.4.2 导入模块


- 这些动作要成为习惯

- 模块版本号

IPython version:7.12.0

numpy version: 1.16.6

scikit-learn version: 0.22.1

In [3]:
%pylab inline

import IPython
import sklearn as sk
import numpy as np
#import matplotlib
#import matplotlib.pyplot as plt

print ('IPython version:', IPython.__version__)
print ('numpy version:', np.__version__)
print ('scikit-learn version:', sk.__version__)
#print ('matplotlib version:', matplotlib.__version__)

Populating the interactive namespace from numpy and matplotlib
IPython version: 7.8.0
numpy version: 1.16.5
scikit-learn version: 0.24.1


### 16.4.3 加载数据

- 加载 20-新闻组数据集（20 newsgroup Dataset）
    - 两种加载方式

#### (1) 加载方式 1 —— 获取20-新闻组数据集


```python
>>> sklearn.datasets.fetch_20newsgroups(
        data_home=None, subset='train', categories=None, shuffle=True, 
        random_state=42, remove=(), download_if_missing=True)
```


- 功能
    - 从20-新闻组数据集，加载文件名和数据


- 参数 —— 略

#### (2) 加载方式 2 —— 获取 20-新闻组 <font color="red">向量化数据集</font>


```python
>>> sklearn.datasets.fetch_20newsgroups_vectorized(subset='train', remove=(), data_home=None, download_if_missing=True)
```


- 功能
    - 加载 20-新闻组数据集


- 参数 —— 略

In [4]:
# datasets 子模块
import sklearn.datasets as ds

In [5]:
# 获取 20-新闻组
news = ds.fetch_20newsgroups(subset='all')

In [6]:
ds.fetch_20newsgroups??

### 16.4.4  观察数据

#### (1) 总览

In [7]:
news

{'data': ["From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>\nSubject: Pens fans reactions\nOrganization: Post Office, Carnegie Mellon, Pittsburgh, PA\nLines: 12\nNNTP-Posting-Host: po4.andrew.cmu.edu\n\n\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n",
  'From: mblawson@midway.ecn.uoknor.edu (Matthew B Lawson)\nSubjec

#### (2) 键

In [8]:
news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

#### (3) 进一步观察数据集


- 字典、关键字
    - data
    - filenames
    - target_names
    - target
    - DESCR
    - description

In [9]:
# 数据、目标、目标名 类型
print (type(news.data), type(news.target), type(news.target_names))

<class 'list'> <class 'numpy.ndarray'> <class 'list'>


In [10]:
news.data[0]

"From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>\nSubject: Pens fans reactions\nOrganization: Post Office, Carnegie Mellon, Pittsburgh, PA\nLines: 12\nNNTP-Posting-Host: po4.andrew.cmu.edu\n\n\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"

In [11]:
# 文件名
news.filenames[0]

'C:\\Users\\solit\\scikit_learn_data\\20news_home\\20news-bydate-test\\rec.sport.hockey\\54367'

In [12]:
# 目标名称列表
news.target_names[news.target[0]]

'rec.sport.hockey'

In [13]:
# 样本数
print (len(news.data))
print (len(news.target))

18846
18846


In [14]:
# 查看 DESCR
print(news.DESCR)

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality

#### (4) 查看第一条样本

- 可见一条新闻组信息，可查到其分类
    - 编号 10
    - 类名 rec.sport.hockey (曲棍球)

In [15]:
# 输出
print (news.target[0])
print (news.target_names[news.target[0]])

10
rec.sport.hockey


In [16]:
# 输出数据
print (news.data[0])

From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




#### (5) 各种类别都看一下吧

In [17]:
for choice in range(20):
    for idx, target in enumerate(news.target):
        if target==choice:
            print("目标类值:{}".format(news.target[idx]))
            print("目标名称:{}".format(news.target_names[target]))
            print("文本序号:{0}\n\n文本内容:\n{1}".format(idx, news.data[idx]))
            print("*"*50)
            break

目标类值:0
目标名称:alt.atheism
文本序号:14

文本内容:
From: kmr4@po.CWRU.edu (Keith M. Ryan)
Subject: Re: Islam And Scientific Predictions (was Re: Genocide is Caused by Atheism)
Organization: Case Western Reserve University
Lines: 14
NNTP-Posting-Host: b64635.student.cwru.edu

In article <1993Apr19.231641.21652@monu6.cc.monash.edu.au> darice@yoyo.cc.monash.edu.au (Fred Rice) writes:

>The positive aspect of this verse noted by Dr. Maurice Bucaille is that
>while geocentrism was the commonly accepted notion at the time (and for
>a long time afterwards), there is no notion of geocentrism in this verse
>(or anywhere in the Qur'an).

	There is no notion of heliocentric, or even galacticentric either.



--------------------------------------------------------------------------------
		
		"My sole intention was learning to fly."

**************************************************
目标类值:1
目标名称:comp.graphics
文本序号:22

文本内容:
From: ruocco@ghost.dsi.unimi.it (sergio ruocco)
Subject: Re: HOT NEW 3D Software
Keyw

## 16.5 训练集-测试集拆分

### 16.5.1 直接分割

- 注意
    - `news.data` 已经是随机排列好的数据集

In [18]:
# 分割长度，按 75% 占比

SPLIT_PERC = 0.75
split_size = int(len(news.data)*SPLIT_PERC)

print("记录总数：{0:>6}\n训练集长：{1:>6}\n测试集长：{2:>6}"
      .format(len(news.data), split_size, len(news.data)-split_size))

记录总数： 18846
训练集长： 14134
测试集长：  4712


In [19]:
# 样本数据 —— 训练集与测试集
X_train = news.data [:split_size]
X_test = news.data  [split_size:]

In [20]:
# 目标数据 —— 训练集与测试集
y_train = news.target [:split_size]
y_test = news.target  [split_size:]

### 16.5.2 交叉验证方法


- K 折
- 均值方差计算模块 scipy.stats

In [21]:
# 旧版本弃用
#from sklearn.cross_validation import cross_val_score, KFold

In [22]:

# 新版本使用
from sklearn.model_selection import cross_val_score, KFold

In [23]:
# 导入计算标准误差函数 sem
from scipy.stats import sem

In [24]:
# 定义交叉验证评价函数
def evaluate_cross_validation(clf, X, y, K):
    # 创建 K-折 交叉验证迭代器（5折）
    #cv = KFold(len(y), K, shuffle=True, random_state=0)
    cv = KFold(K, shuffle=True, random_state=0)
    # 返回得分
    scores = cross_val_score(clf, X, y, cv=cv)
    print (scores)
    print (("平均得分: {0:.3f} (+/-{1:.3f})").format(
        np.mean(scores), sem(scores)))

## 16.6 文本型数据 vs 数值型数据

### 16.6.1 遇到新问题


- 机器学习算法的适用性
    - 仅能用于数值型数据


- 样本数据 —— 单条文本信息
    - 怎么办？

### 16.6.2  解决问题的思路


- 很多词汇具有明显的专业特征（甚至包括符号、数字、标点等）
    - 运动类
    - 经济类
    - 科技类
    - 宗教类
    - 政治类
    - 等


- 考虑根据特定词汇出现的频率分布来进行分类


- Scikit Learn 有专门工具可用来创建数值特性向量

### 16.6.3 文本特征提取工具模块 `sklearn.feature_extraction.text`

In [25]:
# 导入三个类
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

#### (1) 可将 文本信息 转换成 数值特性 的 三个类

- `CountVectorizer` —— 创建一个单词字典，产生一向量，对文本中特定词汇的出现次数进行计数
- `HashingVectorizer` —— 在上述基础上，还实现了一哈希函数，以改善算法效率
- `TfidfVectorizer` —— 工作方式类似于 `CountVectorizer`，不过采用了改进算法，计算称为 `TF-IDF` (<font color="red">词频-逆文档频率</font>)

#### (2) 创建 `CountVectorizer` 对象方法


```python
>>> CountVectorizer(input='content', encoding='utf-8', decode_error='strict',
                    strip_accents=None, lowercase=True, preprocessor=None,
                    tokenizer=None, stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b',
                    ngram_range=(1, 1), analyzer='word',
                    max_df=1.0, min_df=1,
                    max_features=None, vocabulary=None, binary=False,
                    dtype=<class 'numpy.int64'>)
```

- 功能 —— 将原始文档集转换成单词出现个数的特征矩阵


- 参数
    - token_pattern —— 用于匹配特殊词的正则表达式字符串，仅用于 `analyzer == 'word'` 时
    - stop_words —— 英文字符串、列表或 `None`，包含 `stop_words`，即<font color="red">停用词</font>
    - 余略


- <font color="red">停用词</font> stop_words 在自然语言处理中，指的是没有什么实际意义的词语，需要过滤掉，如
    - 中文：啊、的、和、哪、这、那、一等，见 [中文停用词表](https://github.com/elephantnose/characters)
    - 英文：and, or, the, a, an, on, ..., 见 [外文停用词表](https://github.com/Alir3z4/stop-words)

#### (3) 创建 `HashingVectorizer` 对象方法 (适用于旧版本)


```python
>>> HashingVectorizer(input='content', encoding='utf-8', decode_error='strict', strip_accents=None,
                      lowercase=True, preprocessor=None, tokenizer=None, stop_words=None,
                      token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), analyzer='word',
                      n_features=1048576, binary=False, norm='l2', alternate_sign=True,
                      non_negative=False, dtype=<class 'numpy.float64'>)
```


- 功能 —— 将原始文档集转换成单词出现个数的特征哈希矩阵


- 参数
    - token_pattern —— 用于匹配特殊词的正则表达式字符串，仅用于 `analyzer == 'word'` 时
    - stop_words —— 英文字符串、列表或 `None`，包含 `stop_words`，即<font color="red">停用词</font>
    - 余略


- 注意
    - 参数 `non_negative` 新版本将弃用

#### (4) 创建`TfidfVectorizer` 对象方法


```python
>>> TfidfVectorizer(input='content', encoding='utf-8', decode_error='strict', strip_accents=None,
                    lowercase=True, preprocessor=None, tokenizer=None, analyzer='word',
                    stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1),
                    max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False,
                    dtype=<class 'numpy.int64'>, norm='l2', use_idf=True, smooth_idf=True,
                    sublinear_tf=False)
```


- 功能 —— 将原始文档集转换成 `TF-IDF` 特征矩阵


- 参数
    - token_pattern —— 用于匹配特殊词的正则表达式字符串，仅用于 `analyzer == 'word'` 时
    - stop_words —— 英文字符串、列表或 `None`，包含 `stop_words`，即<font color="red">停用词</font>
    - 余略

#### (5) 三个类的区别

- 获取数值特性的功能和算法有所不同

### 16.6.4 词频-逆文档频率 `TF-IDF` 


- TF-IDF —— Term Frequency–Inverse Document Frequency


- 说明
    - 一种统计方法，用来测算单词对于文档的重要性
    - 规避某些过于频繁出现且对于分类没有实质意义的词汇

#### (1) 词频 (TF, Term Frequency)


- 某一词语在该文件中出现的频率，是词数（term count）的归一化

$${\rm tf}_{i,j}=\dfrac{n_{i,j}}{\sum\limits_k{n_{k,j}}}$$

其中，$n_{i,j}$ 是词 $t_i$ 在文档 $d_j$ 中出现的次数，即词数。

#### (2) 逆文档频率 (IDF, Inverse Document Frequency)


- IDF —— 一个词语普遍重要性的度量
    - 总文件数目除以包含该词语之文件的数目，再将得到的商取以10为底的对数

$$
{\rm idf}_i = \log_{10}\dfrac{|D|}{|\rm{count}\{j:t_i\in d_j\}|}
$$


- 其中
    - $|D|$ 为语料库中的文件总数
    - $|\rm{count}\{j:t_i\in d_j\}|$ 为包含词语 $t_{i}$ 的文件数目，即 $n_{i,j}\ne 0$ 的文件数
    - 为避免分母为零，一般可取 $1+|\rm{count}\{j:t_i\in d_j\}|$

$$
{\rm idf}_i = \log_{10}\dfrac{|D|}{1+|\rm{count}\{j:t_i\in d_j\}|}
$$


#### (3) 词频-逆文档频率 `TF-IDF`


$$
{\rm tfidf}_{i,j}={\rm tf}_{i,j}\times {\rm idf_i}
$$


- 高权重的 ${\rm tfidf}_{i,j}$，将由下列因素产生
    - 词 $t_{i}$ 在文件 $d_{j}$ 内的高词频 ${\rm tf}_{i,j}$
    - 词 $t_{i}$ 在整个文件集合中的低<font color="red">逆文档频率</font>


- 因此，tf-idf 适合用于
    - 过滤掉常见的词语
    - 保留重要的词语

## 16.7 训练朴素贝叶斯分类器


- 组合
    - 特征向量化工具
    - 实际贝叶斯分类器

### 16.7.1  `MultinomialNB` 类 —— 特征向量化工具


- 从属模块
    -  `sklearn.naive_bayes`


- 语法
```python
>>> obj = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
```


- 参数
    - alpha 附加光滑参数（0：无光滑，缺省 1：完全光滑）
    - 余略

In [26]:
# 导入 MultinomialNB 类
from sklearn.naive_bayes import MultinomialNB

### 16.7.2 交叉验证分类识别


- 管线 `Pipeline`
    - 按顺序构建一系列转换和一个模型，最后的一步是模型



- 优点 —— 高效率


- 语法
```python
>>> Pipeline(steps, memory=None)
```




#### (1) 利用管线创建三个 `MultinormialNB` 对象

In [27]:
from sklearn.pipeline import Pipeline

clf_1 = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB()),  #光滑化处理
])

In [28]:
clf_2 = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB()),
])

#### (2) 对各分类器进行交叉验证评价

In [29]:
clfs = [clf_1, clf_2]
for clf in clfs:
    evaluate_cross_validation(clf, news.data, news.target, 5)

[0.85782493 0.85725657 0.84664367 0.85911382 0.8458477 ]
平均得分: 0.853 (+/-0.003)
[0.84482759 0.85990979 0.84558238 0.85990979 0.84213319]
平均得分: 0.850 (+/-0.004)


#### (3) 文本特征提取工具 性能比较


- CountVectorizer 和 TfidfVectorizer 比较接近
    - CountVectorizer —— 平均得分 0.853
    - TfidfVectorizer —— 平均得分 0.850

### 16.7.3 调参 —— 改进分类器


- 对 `TfidfVectorizer` 进行改进
- 保留 `TF-IDF` 向量化程序

#### (1) 调整参数 `token_pattern` 


- 参数说明
    - 采用不同的正则表达式来摘取标记词汇
    - 缺省正则表达式为：r"\b\w\w+\b" (正则表达式含义：至少含 2个字母的单词)
        - \w 匹配字母或数字或下划线或汉字 等价于 [A-Za-z0-9_]
        - \s 匹配任意的空白符
        - \d 匹配数字
        - \b 匹配单词的开始或结束
        - ^ 匹配字符串的开始
        - $ 匹配字符串的结束
        - \w 能不能匹配汉字要视你的操作系统和你的应用环境而定


- 思考
    - 文本中多次出现数字或特殊符号
        - 应考虑数字与字符
    - 文本中多次出现网址
        - 可能考虑斜线 `\` 和点字符 `.` 会更好


- <font color="red">新正则表达式为：r"\b[a-z0-9_\\-\\.]+[a-z][a-z0-9_\\-\\.]+\b"</font>


- 关于正则表达式的知识
    - 详细内容参考模块 `re` 的文档

#### (2) 创建调参后的分类器对象

- 所调参数 —— `token_pattern`

In [30]:
clf_3 = Pipeline([
    ('vect', TfidfVectorizer(token_pattern = r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",)),
    ('clf', MultinomialNB())
])

#### (3) 对 `clf_3` 交叉验证评价

- 效果得到改进，得分 0.865

In [31]:
evaluate_cross_validation(clf_3, news.data, news.target, 5)

[0.86100796 0.8718493  0.86203237 0.87291059 0.8588485 ]
平均得分: 0.865 (+/-0.003)


#### (4) 调整参数 `stop_words`


- 参数功能 —— 允许传递一个不打算计入的词汇列表，含
    - 过于频繁出现的词汇（如 the, a, an, ok, oh, and, or,......）
    - 先验词汇（预知其不能提供帮助）

In [32]:
# 定义函数
def get_stop_words():
    result = set()
    for line in open('stopwords_en.txt', 'r').readlines():
        result.add(line.strip())
    return result

In [33]:
# 返回 stop_words
stop_words = get_stop_words()
print ("{0}个停用词\n\n停用词集合为\n{1}".format(len(stop_words),stop_words))

318个停用词

停用词集合为
{'she', 'show', 'interest', 'do', 'along', 'fill', 'wherever', 'became', 'third', 'formerly', 'full', 'therein', 'no', 'bill', 'fifteen', 'thin', 'to', 'anyhow', 'made', 'con', 'whole', 'latterly', 'whereupon', 'further', 'ie', 'otherwise', 'across', 'such', 'however', 'part', 'last', 'them', 'there', 'see', 'wherein', 'after', 'and', 'anyway', 'front', 'or', 'all', 'being', 'whenever', 'from', 'each', 'by', 'for', 'twelve', 'thus', 'nevertheless', 'without', 'nothing', 'always', 'moreover', 'we', 'whose', 'down', 'yourself', 'whereby', 'keep', 'you', 'several', 'top', 'seemed', 'may', 'nowhere', 'him', 'about', 'fify', 'other', 'have', 'now', 'twenty', 'sixty', 'because', 'were', 'someone', 'something', 'will', 'very', 'me', 'amount', 'whereas', 'often', 'move', 'per', 'somewhere', 'another', 'as', 'of', 'more', 'sometime', 'also', 'indeed', 'itself', 'been', 'on', 'herself', 'many', 'name', 'hereupon', 'latter', 'until', 'inc', 'are', 'none', 'could', 'thereby', 'off'

#### (5) 创建进一步调参后的分类器对象

In [34]:
# 改进后的分类器
clf_4 = Pipeline([
    ('vect', TfidfVectorizer(
                stop_words=stop_words,
                token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",    
    )),
    ('clf', MultinomialNB()),
])

#### (6) 对 `clf_4` 交叉验证评价

- 得分进一步提高

In [35]:
# 检验分类器
evaluate_cross_validation(clf_4, news.data, news.target, 5)

[0.88116711 0.89519767 0.88325816 0.89227912 0.88113558]
平均得分: 0.887 (+/-0.003)


#### (7)  调整 `MultinomialNB` 分类器中的参数 `alpha`


- 取 alpha = 0.01

In [36]:
clf_5 = Pipeline([
    ('vect', TfidfVectorizer(
                stop_words=stop_words,
                token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",         
    )),
    ('clf', MultinomialNB(alpha=0.01)),
])

#### (8)  对 `clf_5` 交叉验证评价

- 得分达到 0.921
- 效果显著提高

In [37]:
evaluate_cross_validation(clf_5, news.data, news.target, 5)

[0.9204244  0.91960732 0.91828071 0.92677103 0.91854603]
平均得分: 0.921 (+/-0.002)


#### (9) 调参 —— 没有止境


- 可以试试对向量分类器 `CountVectorizer` 进行调参
    - stop_words
    - token_pattern

### 16.7.4 开始测试


- 前述多次尝试
    - 三个 `vectorizer` 比较
    - 调参
    - 改进了分类器


- 交叉验证最高得分达到 0.921，分类器是 `clf_5`


- 接受了 `clf_5`


- 开始测试...

#### (1) 定义函数


- 函数语法
```python
>>> train_and_evaluate(clf, X_train, X_test, y_train, y_test)
```

- 功能 —— 训练与评价


- 参数 （略）

In [38]:
from sklearn import metrics

def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    
    clf.fit(X_train, y_train)
    
    print ("训练集精度 Accuracy：")
    print (clf.score(X_train, y_train))
    print ("测试集精度 Accuracy：")
    print (clf.score(X_test, y_test))
    
    y_pred = clf.predict(X_test)
    
    print ("分类测试报告：")
    print (metrics.classification_report(y_test, y_pred))
    print ("混淆矩阵：")
    print (metrics.confusion_matrix(y_test, y_pred))

#### (2) 将最佳效果的 `clf_5` 投入实用

In [39]:
train_and_evaluate(clf_5, X_train, X_test, y_train, y_test)

训练集精度 Accuracy：
0.9969576906749682
测试集精度 Accuracy：
0.9178692699490663
分类测试报告：
              precision    recall  f1-score   support

           0       0.95      0.88      0.91       216
           1       0.85      0.85      0.85       246
           2       0.91      0.84      0.87       274
           3       0.81      0.86      0.83       235
           4       0.88      0.90      0.89       231
           5       0.89      0.91      0.90       225
           6       0.88      0.80      0.84       248
           7       0.92      0.93      0.93       275
           8       0.96      0.98      0.97       226
           9       0.97      0.94      0.96       250
          10       0.97      1.00      0.98       257
          11       0.97      0.97      0.97       261
          12       0.90      0.91      0.91       216
          13       0.94      0.95      0.95       257
          14       0.94      0.97      0.95       246
          15       0.90      0.96      0.93       234
   

#### (3) 结果分析


- 上述训练和测试表明
    - 获得了很好的结果分数（训练集 0.997，测试集 0.918），可以接受该模型
    - 训练集的表现，好于测试集


- 对所训练的分类器预期
    - 测试未知数据 —— 得分可以达到 0.91

### 16.7.5 进一步观察训练好的 `vectorizer`

- 对字典有典型影响的标记符号
    - `feature_names`


- 其它

In [40]:
# 观察训练后，clf_5 的属性和方法
clf_5namestep=clf_5.named_steps['vect']
dir(clf_5namestep)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_char_ngrams',
 '_char_wb_ngrams',
 '_check_n_features',
 '_check_params',
 '_check_stop_words_consistency',
 '_check_vocabulary',
 '_count_vocab',
 '_get_param_names',
 '_get_tags',
 '_limit_features',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_sort_features',
 '_stop_words_id',
 '_tfidf',
 '_validate_data',
 '_validate_params',
 '_validate_vocabulary',
 '_warn_for_unused_params',
 '_white_spaces',
 '_word_ngrams',
 'analyzer',
 'binary',
 'build_analyzer',
 'build_preprocessor',
 'build_tokenizer',
 'decode',
 'decode_error',
 'dtype',
 'encoding',
 'f

In [41]:
# 查看 feature_names
feature_names=clf_5.named_steps['vect'].get_feature_names()
print(feature_names)

['...and', '0-.66d8wt', '0-04g55', '0-100mph', '0-13-117441-x--or', '0-3mb', '0-40mb', '0-40volts', '0-5mb', '0-60mph', '0-8.3mb', '0-a00138', '0-byte', '0-defects', '0-e8', '0-for-4', '0-hc', '0-ii', '0-uw', '0-uw0', '0-uw2', '0-uwa', '0-uwt', '0-uwt7', '0-uww', '0-uww7', '0.-w0', '0..x-1', '0.00...nice', '0.02cents', '0.0cb', '0.1-ports', '0.15mb', '0.2d-_', '0.5db', '0.6-micron', '0.65mb', '0.97pl4', '0.b34s_', '0.c0rgo5kj7pp0', '0.c4', '0.jy', '0.s_', '0.tprv6ekj7r', '0.tt', '0.txa_', '0.txc', '0.vpp', '0.vpsll2', '00-index.txt', '000-foot', '000-kg', '000-man', '000-maxwell', '000-strong', '000000.active.spx', '000062david42', '000100255pixel', '0005111312na1em', '0005111312na3em', '000hz', '000iu', '000mg', '000mi', '000miles', '000puq9', '000rpm', '000th', '000ug', '000usd', '0010580b.0b6r49', '0010580b.vma7o9', '0010580b.vmcbrt', '001200201pixel', '002251w.5.734117130', '007bww3', '007gjf3', '00969fba.e640ff10', '0096b0f0.c5de05a0', '0096b11b.08a283a0', '0096b294.aad9c1e0', '00

In [42]:
# 查看有多少个特征
print (len(clf_5.named_steps['vect'].get_feature_names()))

145767


### 第16讲 结束